<a href="https://colab.research.google.com/github/lucascarneirodev/Data-Science-Projects/blob/main/Mod3_Projeto_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exercício
Neste projeto você deverá desenvolver a partir do que já vimos nos projetos anteriores, ou seja, poderá usar qualquer um dos modelos vistos durante as aulas. Siga este roteiro e preencha cada parte.

In [3]:
%pip install -U scikit-learn pandas numpy seaborn

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LassoCV, RidgeCV


from warnings import simplefilter

In [4]:
data = pd.read_csv("train.csv")
data_train = pd.read_csv("test.csv")

In [5]:
# referência do código: João Rodrigues
def print_metrics(model: Pipeline, model_name: str, X_test: np.ndarray, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE) and R-squared scores of the model
      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA
      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.
  """
  #imports
  from sklearn.metrics import mean_absolute_error, mean_squared_error

  return (f'''

Mean Absolute Error (MAE) do Modelo {model_name}: {mean_absolute_error(y_test, y_pred):.3f}

Mean Squared Error (MSE) do Modelo {model_name}: {mean_squared_error(y_test, y_pred, squared=True):.4f}

Root Mean Squared Error (RMSE) do Modelo {model_name}: {mean_squared_error(y_test, y_pred, squared=False):.4f}

R-squared do Modelo {model_name}: {model.score(X_test, y_test):.3f}
''')

def get_features(model: Pipeline, model_name: str, ispca: bool) -> str:
  """
  Get Features Function
  Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
      Arguments:

          - model {Pipeline} -- a pipeline object containing an estimator/model with fit/transform/predict operations.

          - model_name {str} -- name of the model used inside the pipeline to identify the model

          - ispca {bool} -- if the model passed is a model with pca, this should be true, otherwise it should be false.
      Returns:
          Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
  """

  preprocessor_fitted = model['preprocessor']

  if ispca:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['pca'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['pca'].get_feature_names_out())
      )

  else:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
      )

  feature_names = np.array(feature_names)
  kept = ~np.isclose(model[model_name].coef_, 0)

  if 'lasso' not in model_name:
    return(f'''
    Atenção: O modelo passado não é Lasso, logo todas as variáveis passadas terão sido selecionadas.
    Número de Features Original: {len(feature_names)}
    Número de Features selecionadas pelo {model_name}: {len(feature_names[kept])}
    Features Selecionadas:
    {feature_names[kept]}
    ''')

  return(f'''
    Número de Features Original: {len(feature_names)}
    Número de Features selecionadas pelo {model_name}: {len(feature_names[kept])}
    Features Selecionadas:
    {feature_names[kept]}
    ''')

In [6]:
simplefilter("ignore", category=UserWarning)

#1. Análise exploratória de dados (1.0 ponto)

Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#2. Levantamento de hipóteses (1.0 ponto)
Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.

#3. Apresentação das ideias obtidas (1.0 ponto)
Apresente com gráficos as suas observações e a descreva cada gráfico.

#4. Preparação dos dados (1.0 ponto)
Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados

In [38]:
col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

numeric_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
        ]
)
categorical_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
        ]
)
preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer, col_num),
    ("cat", categorical_transformer, col_tex)
  ]
)
modelLassoCV = Pipeline(
  steps=[
    ("preprocessor", preprocessor),
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59D71F0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59D7280>)])),
                ('lassocv', LassoCV(cv=5))])

In [10]:
modelLassoCV.fit(X_train, y_train)
y_pred = modelLassoCV.predict(X_test)

preprocessor_fitted = modelLassoCV['preprocessor']

feature_names = (
  list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
  list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
)

feature_names = np.array(feature_names)

kept = ~np.isclose(modelLassoCV['lassocv'].coef_, 0)

print(f'''
Número de Features Original: {len(feature_names)}
Número de Features selecionadas pelo LassoCV: {len(feature_names[kept])}
Features Selecionadas:
{feature_names[kept]}
''')


Número de Features Original: 300
Número de Features selecionadas pelo LassoCV: 90
Features Selecionadas:
['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'TotalBsmtSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath'
 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr'
 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars' 'GarageArea'
 'WoodDeckSF' 'OpenPorchSF' '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal'
 'MoSold' 'x0_RM' 'x3_IR2' 'x4_Bnk' 'x4_HLS' 'x6_CulDSac' 'x7_Mod'
 'x8_BrkSide' 'x8_Crawfor' 'x8_Edwards' 'x8_Gilbert' 'x8_Mitchel'
 'x8_NAmes' 'x8_NWAmes' 'x8_NoRidge' 'x8_NridgHt' 'x8_Somerst'
 'x8_StoneBr' 'x9_Norm' 'x10_PosN' 'x11_1Fam' 'x11_2fmCon' 'x12_1Story'
 'x12_2Story' 'x13_Gable' 'x14_ClyTile' 'x15_BrkFace' 'x15_HdBoard'
 'x16_ImStucc' 'x16_Plywood' 'x17_BrkFace' 'x17_None' 'x18_Ex' 'x18_TA'
 'x20_BrkTil' 'x20_PConc' 'x21_Ex' 'x22_TA' 'x23_Gd' 'x23_No' 'x24_G

In [11]:
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV: 17523.065

Mean Squared Error (MSE) do Modelo Pipeline LassoCV: 866285046.5730

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV: 29432.7207

R-squared do Modelo Pipeline LassoCV: 0.887



#5. Criação e seleção de características (1.0 ponto)
Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?

Criando pipeline utilizando o pca.

In [12]:
col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelLassoCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1FEF88BB0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1FEF8BE50>)])),
                ('lassocv', LassoCV(cv=5))])

In [16]:
modelLassoCV_pca.fit(X_train, y_train)
y_pred = modelLassoCV_pca.predict(X_test)

print(get_features(model=modelLassoCV, model_name='lassocv', ispca= False))

print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

preprocessor_fitted = modelLassoCV_pca['preprocessor']

feature_names = (
  list(preprocessor_fitted.named_transformers_['num']['pca'].get_feature_names_out()) +
  list(preprocessor_fitted.named_transformers_['cat']['pca'].get_feature_names_out())
)

feature_names = np.array(feature_names)

kept = ~np.isclose(modelLassoCV_pca['lassocv'].coef_, 0)

print(f'''
Número de Features Original: {len(feature_names)}
Número de Features selecionadas pelo LassoCV: {len(feature_names[kept])}
Features Selecionadas:
{feature_names[kept]}
''')


    Número de Features Original: 300
    Número de Features selecionadas pelo lassocv: 90
    Features Selecionadas:
    ['Id' 'MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'TotalBsmtSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath'
 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr'
 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars' 'GarageArea'
 'WoodDeckSF' 'OpenPorchSF' '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal'
 'MoSold' 'x0_RM' 'x3_IR2' 'x4_Bnk' 'x4_HLS' 'x6_CulDSac' 'x7_Mod'
 'x8_BrkSide' 'x8_Crawfor' 'x8_Edwards' 'x8_Gilbert' 'x8_Mitchel'
 'x8_NAmes' 'x8_NWAmes' 'x8_NoRidge' 'x8_NridgHt' 'x8_Somerst'
 'x8_StoneBr' 'x9_Norm' 'x10_PosN' 'x11_1Fam' 'x11_2fmCon' 'x12_1Story'
 'x12_2Story' 'x13_Gable' 'x14_ClyTile' 'x15_BrkFace' 'x15_HdBoard'
 'x16_ImStucc' 'x16_Plywood' 'x17_BrkFace' 'x17_None' 'x18_Ex' 'x18_TA'
 'x20_BrkTil' 'x20_PConc' 'x21_Ex' 'x22_TA' 'x23_Gd'

In [14]:
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV: 18956.904

Mean Squared Error (MSE) do Modelo Pipeline LassoCV: 978705610.2881

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV: 31284.2710

R-squared do Modelo Pipeline LassoCV: 0.872



##Pipeline com Ridge e pca

In [24]:
col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor_pca = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelLassoCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor_pca), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59CACE0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59CAF50>)])),
                ('lassocv', LassoCV(cv=5))])

In [32]:
modelLassoCV_pca.fit(X_train, y_train)
y_pred = modelLassoCV_pca.predict(X_test)

In [33]:
print(get_features(model=modelLassoCV_pca, model_name='lassocv', ispca= True))


    Número de Features Original: 250
    Número de Features selecionadas pelo lassocv: 108
    Features Selecionadas:
    ['pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6' 'pca7' 'pca8' 'pca9'
 'pca10' 'pca11' 'pca14' 'pca15' 'pca16' 'pca17' 'pca18' 'pca19' 'pca20'
 'pca22' 'pca23' 'pca24' 'pca25' 'pca26' 'pca27' 'pca28' 'pca29' 'pca31'
 'pca32' 'pca33' 'pca34' 'pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6'
 'pca7' 'pca8' 'pca9' 'pca10' 'pca12' 'pca13' 'pca14' 'pca15' 'pca16'
 'pca17' 'pca18' 'pca19' 'pca20' 'pca22' 'pca23' 'pca24' 'pca25' 'pca26'
 'pca27' 'pca29' 'pca32' 'pca33' 'pca34' 'pca35' 'pca36' 'pca39' 'pca40'
 'pca41' 'pca42' 'pca43' 'pca44' 'pca45' 'pca47' 'pca48' 'pca51' 'pca52'
 'pca53' 'pca55' 'pca56' 'pca57' 'pca59' 'pca60' 'pca61' 'pca62' 'pca64'
 'pca67' 'pca68' 'pca69' 'pca71' 'pca75' 'pca76' 'pca78' 'pca82' 'pca85'
 'pca86' 'pca88' 'pca89' 'pca94' 'pca96' 'pca98' 'pca99' 'pca102' 'pca104'
 'pca106' 'pca107' 'pca118' 'pca131' 'pca180' 'pca182' 'pca184']
    


In [34]:
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV com PCA: 18956.904

Mean Squared Error (MSE) do Modelo Pipeline LassoCV com PCA: 978705610.2881

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV com PCA: 31284.2710

R-squared do Modelo Pipeline LassoCV com PCA: 0.872



In [31]:
#Testando pipeline utilizando Ridge e pca

modelRidgeCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor_pca),
    ("ridgecv", RidgeCV(cv=5))
  ]
)

modelRidgeCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59CACE0>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001D1A59CAF50>)])),
                ('ridgecv', RidgeCV(cv=5))])

In [35]:
modelRidgeCV_pca.fit(X_train, y_train)
y_pred = modelRidgeCV_pca.predict(X_test)

In [36]:
print(get_features(model=modelRidgeCV_pca, model_name='ridgecv', ispca= True))


    Atenção: O modelo passado não é Lasso, logo todas as variáveis passadas terão sido selecionadas.
    Número de Features Original: 250
    Número de Features selecionadas pelo ridgecv: 250
    Features Selecionadas:
    ['pca0' 'pca1' 'pca2' 'pca3' 'pca4' 'pca5' 'pca6' 'pca7' 'pca8' 'pca9'
 'pca10' 'pca11' 'pca12' 'pca13' 'pca14' 'pca15' 'pca16' 'pca17' 'pca18'
 'pca19' 'pca20' 'pca21' 'pca22' 'pca23' 'pca24' 'pca25' 'pca26' 'pca27'
 'pca28' 'pca29' 'pca30' 'pca31' 'pca32' 'pca33' 'pca34' 'pca0' 'pca1'
 'pca2' 'pca3' 'pca4' 'pca5' 'pca6' 'pca7' 'pca8' 'pca9' 'pca10' 'pca11'
 'pca12' 'pca13' 'pca14' 'pca15' 'pca16' 'pca17' 'pca18' 'pca19' 'pca20'
 'pca21' 'pca22' 'pca23' 'pca24' 'pca25' 'pca26' 'pca27' 'pca28' 'pca29'
 'pca30' 'pca31' 'pca32' 'pca33' 'pca34' 'pca35' 'pca36' 'pca37' 'pca38'
 'pca39' 'pca40' 'pca41' 'pca42' 'pca43' 'pca44' 'pca45' 'pca46' 'pca47'
 'pca48' 'pca49' 'pca50' 'pca51' 'pca52' 'pca53' 'pca54' 'pca55' 'pca56'
 'pca57' 'pca58' 'pca59' 'pca60' 'pca61' 'pca62' '

In [37]:
print(f'Pontuação do Modelo: {print_metrics(model=modelRidgeCV_pca, model_name="Pipeline RidgeCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline RidgeCV com PCA: 19171.747

Mean Squared Error (MSE) do Modelo Pipeline RidgeCV com PCA: 934734415.2342

Root Mean Squared Error (RMSE) do Modelo Pipeline RidgeCV com PCA: 30573.4266

R-squared do Modelo Pipeline RidgeCV com PCA: 0.878



#6. Decisão do modelo (1.0 ponto)
Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?

#7. Modelagem (1.0 ponto)
Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.

#8. Histórico (1.0 pontos)
Preencha a tabela abaixo com cada teste realizado com seu modelo (coloque vazio se a métrica não se aplica). Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.

#9. Resultado (2.0 pontos)
Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.